# Report

## Queue comparison

CLASS QUEUES (from lecture) vs STOCK MARKET QUEUES (LOB)

CLASS QUEUES (M/M/1, G/G/1, M/M/c, etc.)
- “Service” queues: jobs arrive → wait → get served → depart
- Defined by A/B/c/N/K notation (arrival dist., service dist., servers, capacity)
- FIFO or simple disciplines (FIFO, LIFO, priority, random)
- Key parameters: λ (arrival rate), μ (service rate), ρ (utilization)
- Require a server with a service time distribution
- Usually one queue line (or c identical parallel servers)
- Goals: compute W, Wq, L, Lq, stability (ρ < 1)
- No notion of price; all jobs are homogeneous

STOCK MARKET QUEUES (Bid/Ask LOB)
- Not service queues; orders do not receive “service” but wait for matching or cancellation
- Two sides: Bid (max-price priority) and Ask (min-price priority)
- One FIFO queue per price level; many queues in parallel
- Matching happens when buy price ≥ sell price; no service rate μ
- Strict priority: price priority first, FIFO within the same price level
- Orders may be partially filled, canceled, or modified at any time
- Market orders bypass queues and execute immediately
- System is event-driven, not service-time-driven
- Heterogeneous orders: limit, market, cancel, modify, each with price and quantity

KEY DIFFERENCES
- Class queues are time-driven service models; LOB is an event-driven matching system
- Class queues use service rates μ; LOB has no service times or servers
- Class queues have one main queue; LOB has many price-level queues with priority
- Class queues use FIFO; LOB uses price priority → FIFO
- Class queues rely on steady-state formulas; LOB follows matching rules
